In [1]:
from pyomo.environ import *
from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models.pressure_changer import Pump
from idaes.generic_models.properties import iapws95
from idaes.generic_models.properties.helmholtz.helmholtz import (PhaseType)
from idaes.core.util.model_statistics import degrees_of_freedom
import pytest

solver = SolverFactory('ipopt')
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = iapws95.Iapws95ParameterBlock(default={"phase_presentation":PhaseType.L})
m.fs.unit = Pump (default={
    "property_package": m.fs.properties})


In [2]:
# m.y1 = Var(within = Binary, doc = '1 pump 1 is on, 0 is off')
# Q = Var (within = NonNegativeReals, bounds=(24, 32))
# Q_constraint = Constraint (expr = Q == 120*m.y1)

Q = 24 # mol/s
m.fs.unit.BHP = Var (within = NonNegativeReals)

P_disc_constraint = Constraint (expr = m.fs.unit.outlet.pressure[0] == 
                     ((-3.1795*Q**4)+(150.43*Q**3)-(2258.8*Q**2)*(22975*Q)+3000000))
eff_constraint = Constraint (expr = m.fs.unit.efficiency_pump ==  
                   (0.00002*Q**3)-(0.0022*Q**2)+(0.0774*Q)+(0.009))
BHP = Constraint (expr = m.fs.unit.BHP ==  
                   (-0.021*Q**3)+(0.9103*Q**2)+(0.0248*Q)+(408.11))

In [3]:
print (value(m.fs.unit.outlet.pressure[0]))

100000.0


In [4]:
print  ((-3.1795*Q**4)+(150.43*Q**3)-(2258.8*Q**2)-(22975*Q)+3000000)

2172193.728


In [ ]:
Tin = 350  # K
Pin = 101325  # Pa
hin = value(iapws95.htpx(Tin*units.K, Pin*units.Pa))

# set inputs
m.fs.unit.inlet.flow_mol[0].fix(Q)  # mol/s
m.fs.unit.inlet.enth_mol[0].fix(hin)
m.fs.unit.inlet.pressure[0].fix(Pin)
# m.fs.unit.outlet.pressure[0].fix(P_des)
# m.fs.unit.efficiency_pump.fix(eff_P)


# assert value(m.fs.unit.efficiency_isentropic[0]) == pytest.approx(0.9, rel=1e-3)
# assert value(m.fs.unit.deltaP[0]) == pytest.approx(-3e5, rel=1e-3)

In [ ]:
print (degrees_of_freedom(m)) 

In [ ]:
m.fs.unit.initialize()
solver.solve(m, tee=False)

In [ ]:
m.fs.unit.report()